In [26]:
import torch
#GPU使用可能確認
print(torch.cuda.is_available())
#使用できるデバイス数確認
print(torch.cuda.device_count())

True
2


In [27]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sun Jul 30 15:40:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:81:00.0  On |                  N/A |
| 48

In [28]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DebertaTokenizer, DebertaForSequenceClassification, AdamW, AutoTokenizer, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import MeCab


In [30]:
# 形態素解析器の初期化
m = MeCab.Tagger("-Owakati")

単語のベクトル化（ID化）はAutoTokenizerのencode_plusメソッドを使用して行われている。
具体的にはTextClassificationDatasetクラスの__getitem__メソッド内で行われる。
以下参照

In [31]:
"""
encoding = self.tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=self.max_len,
    return_token_type_ids=False,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
)
"""

"\nencoding = self.tokenizer.encode_plus(\n    text,\n    add_special_tokens=True,\n    max_length=self.max_len,\n    return_token_type_ids=False,\n    padding='max_length',\n    return_attention_mask=True,\n    return_tensors='pt',\n    truncation=True\n)\n"

In [32]:
# データセットクラスの定義
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # MeCabを用いて形態素解析
        text = m.parse(text)
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }



In [33]:
# GPUの設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [34]:
# 評価指標の計算
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

### DeBERTa V3のトークナイザーは特定のものを指定せず、モデルの名前を直接使用
https://huggingface.co/microsoft/deberta-v3-baseを用いてpre-traind 
また、gitに公開しているコードはこちらhttps://github.com/microsoft/DeBERTa


In [35]:
from transformers import DebertaConfig, DebertaForSequenceClassification, AutoTokenizer,AutoModelForSequenceClassification
# トークナイザーの初期化
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/sakulab/anaconda3/envs/[yaguchi]/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [36]:
# データの読み込み
train_data = pd.read_csv('/home/sakulab/workspace/yaguchi/research/Master/data/意見データ(5分割交差検証用)/train_data(5).csv')  # 訓練データのパス
val_data = pd.read_csv('/home/sakulab/workspace/yaguchi/research/Master/data/意見データ(5分割交差検証用)/val_data(5).csv')  # バリデーションデータのパス
test_data = pd.read_csv('/home/sakulab/workspace/yaguchi/research/Master/data/意見データ(5分割交差検証用)/test_data(5).csv')  # テストデータのパス

In [37]:
# データの前処理
train_texts = train_data['text'].tolist()
train_labels = train_data['label'].tolist()
test_texts = test_data['text'].tolist()
test_labels = test_data['label'].tolist()
val_texts = val_data['text'].tolist()
val_labels = val_data['label'].tolist()


In [38]:
# データセットの作成
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_len=128)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_len=128)
test_dataset = TextClassificationDataset(test_texts, test_labels, tokenizer, max_len=128)

In [39]:
# モデルの初期化
model = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-base', num_labels=len(set(train_labels)))


Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'mask_predictions.classifier.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

In [40]:
model = model.to(device)

In [41]:
tokenizer.tokenize('中野キャンパスは狭い')

['▁', '中', '野', 'キ', 'ャ', 'ン', 'パ', 'ス', 'は', '狭', 'い']

In [43]:

# トレーニングの設定
training_args = TrainingArguments(
    output_dir='./results/checkpoint_val_5(epoch10)',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs/val_5(epoch10)',
    logging_steps=10,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

In [44]:
# 早期停止の設定
early_stopping_rounds = 5
no_improve_in_last_n_rounds = 0
best_val_loss = np.inf

### 最適化関数：AdamW

In [45]:
#最適化関数の設定
from transformers import AdamW

class CustomTrainer(Trainer):
    def create_optimizer_and_scheduler(self, num_training_steps: int):
        self.optimizer = AdamW(
            self.model.parameters(),
            lr=self.args.learning_rate
        )
        self.lr_scheduler = get_scheduler(
            "linear",
            optimizer=self.optimizer,
            num_warmup_steps=self.args.warmup_steps,
            num_training_steps=num_training_steps
        )

In [46]:
# トレーナーの初期化
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# モデルの訓練
trainer.train()

# モデルの評価
trainer.evaluate(test_dataset)

/home/sakulab/anaconda3/envs/[yaguchi]/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/sakulab/anaconda3/envs/[yaguchi]/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
10,0.671900,0.668047,0.746988,0.000000,0.000000,0.000000
20,0.660900,0.657203,0.746988,0.000000,0.000000,0.000000
30,0.664000,0.639275,0.746988,0.000000,0.000000,0.000000
40,0.622900,0.609527,0.746988,0.000000,0.000000,0.000000
50,0.590600,0.574740,0.746988,0.000000,0.000000,0.000000
60,0.571400,0.551309,0.746988,0.000000,0.000000,0.000000
70,0.496700,0.527222,0.746988,0.000000,0.000000,0.000000
80,0.463500,0.528014,0.746988,0.000000,0.000000,0.000000
90,0.566100,0.527951,0.746988,0.000000,0.000000,0.000000
100,0.533500,0.515209,0.746988,0.000000,0.000000,0.000000


/home/sakulab/anaconda3/envs/[yaguchi]/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sakulab/anaconda3/envs/[yaguchi]/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/sakulab/anaconda3/envs/[yaguchi]/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sakulab/anaconda3/envs/[yagu

{'eval_loss': 0.5026600956916809,
 'eval_accuracy': 0.7937937937937938,
 'eval_f1': 0.5242494226327944,
 'eval_precision': 0.5958005249343832,
 'eval_recall': 0.46804123711340206,
 'eval_runtime': 6.3905,
 'eval_samples_per_second': 312.651,
 'eval_steps_per_second': 2.504,
 'epoch': 10.0}

In [47]:
# ベストモデルのロード
best_model = AutoModelForSequenceClassification.from_pretrained('/home/sakulab/workspace/yaguchi/research/Master/code/LLM/Deberta/results/checkpoint_val_5(epoch10)/checkpoint-2000')

# ベストモデルを使用したトレーナーの初期化
best_trainer = Trainer(
    model=best_model,
    args=training_args,
    compute_metrics=compute_metrics
)

# ベストモデルの評価
best_eval_result = best_trainer.evaluate(test_dataset)

# 評価結果の表示
for key, value in best_eval_result.items():
    print(f"{key}: {value}")

/home/sakulab/anaconda3/envs/[yaguchi]/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


eval_loss: 0.9117526412010193
eval_accuracy: 0.7922922922922923
eval_f1: 0.5532831001076427
eval_precision: 0.5788288288288288
eval_recall: 0.5298969072164949
eval_runtime: 6.34
eval_samples_per_second: 315.142
eval_steps_per_second: 2.524


In [1]:
!nvidia-smi

Sun Jul 30 16:58:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:81:00.0  On |                  N/A |
| 46%   52C    P8    40W / 350W |    349MiB / 24576MiB |      7%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:C1:00.0 Off |                  N/A |
| 40%   

In [2]:
#GPUのメモリ解放
! kill -9 $(lsof -t /dev/nvidia*)